[View in Colaboratory](https://colab.research.google.com/github/SURESHKUMARP1985/ExData_Plotting1/blob/master/dogs_cats_CNN_2.ipynb)

In [0]:
##### We need the imutils library which is not default in google colab
!pip install imutils

In [0]:
!rm -r data

In [0]:
##### downlaod the dogs and cats dataset and save it to a local directory called data
!mkdir data && wget http://files.fast.ai/data/dogscats.zip && unzip dogscats.zip -d data/

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from imutils import paths
import keras
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

In [0]:
#### Extract a list of image file names from the directory to visualise
PATH = 'data/dogscats/'
#### run the list files command and get the top results and save it into a list object
files = !ls {PATH}valid/cats | head 
print(files)

In [0]:
#### Open and Visualise a Single Image
img_path = PATH +'valid/cats/'+files[1]
img = cv2.imread(img_path)
plt.imshow(img)
plt.show()

In [0]:
#### Lets intialise some list variables
data = []
labels = []
dataset_path = 'data/dogscats/train'

#### the imutils paths.list_images function will return the full path of all images
#### in the current directory and its sub-directories
imagePaths = list(paths.list_images(dataset_path))

In [0]:
##### Iterate through all the imagepaths generated previously and load every image
##### figure out the label of the image from its path

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    
    #### Extract the label of the image from the image path
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
   
    #### Resize the Image and then use the raw pixel values as features
    features = cv2.resize(image, (32, 32))
    
    #### Append the features and labels to our list variables
    data.append(features)
    labels.append(label)
    
    # show an update every 1,000 images
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

In [0]:
##### Print the labels of different objects to see they are different
print(labels[0])
print(labels[17000])
print(data[7000].shape)

In [0]:
#### Encode the labels to Numerical Values
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels[0])

#### One Hot Encode the Numerical Values
labels = np_utils.to_categorical(labels, 2)
print(labels[0])

#### Scale the input pixel values
data = np.array(data) / 255.0


In [0]:
#### Create a train test split
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [0]:
# input image dimensions
input_shape = data[0].shape
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
##### Intialise a Stochastic Gradient Descent Optimiser Object
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [0]:
#### Train the Model
model.fit(trainData, trainLabels, epochs=150, batch_size=64)

In [0]:
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=64, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

In [0]:
##### To Predict on a New Unseen Image
##### Step1 Laod an unseen image
image = cv2.imread("data/dogscats/valid/cats/cat.8519.jpg")
plt.imshow(image)
plt.show()

In [0]:
##### Flatten the test image
test_feature = cv2.resize(image, (32, 32))
print(test_feature.shape)

##### Reshape the test image shape to match the network thats expecting a set of images
test_feature = test_feature.reshape(-1,test_feature.shape[0],test_feature.shape[1],test_feature.shape[2])
print(test_feature.shape)


In [0]:
##### Print Predictions
test_prediction = model.predict(test_feature)
print("[INFO] CAT={:.10f}%, DOG: {:.10f}%".format(test_prediction[0][0]*100, test_prediction[0][1]*100))

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#2. Get the file
downloaded = drive.CreateFile({'id':'19PbbBVytGKdmMgeOmqRsXhceGLZGdzci'}) # replace the id with id of file you want to access
downloaded.GetContentFile('dog.jpeg')  
downloaded = drive.CreateFile({'id':'1SboBUMGeAV0eIJgQBESKWsLgOl3o-rgT'}) # replace the id with id of file you want to access
downloaded.GetContentFile('dog2.jpeg')  

In [0]:
!wget https://www.petmd.com/sites/default/files/shutterstock_54135856-slide2.jpg -O cat1.jpg
!ls

In [0]:
##### To Predict on a New Unseen Image
##### Step1 Laod an unseen image
#image = cv2.imread("data/dogscats/valid/cats/cat.10016.jpg")

image = cv2.imread("cat1.jpg")
plt.imshow(image)
plt.show()


In [0]:
##### Preprocess the test image
test_feature = cv2.resize(image, (32, 32))

print(test_feature.shape)
print(test_feature.shape)

test_feature = test_feature.reshape(-1,test_feature.shape[0],test_feature.shape[1],test_feature.shape[2])

pr = model.predict_classes(test_feature)
pro = model.predict_proba(test_feature)
print(pr, pro)

In [0]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("fruits_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [0]:
!ls
from google.colab import files
files.download('model.json') 

In [0]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")